<a href="https://colab.research.google.com/github/FeraMaks/test/blob/main/08_LogisticRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries

In [1]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
#import matplotlib.pyplot as plt


# Initialisation

In [10]:
bc = datasets.load_breast_cancer()

X, Y = bc.data, bc.target

n_samples, n_features = X.shape

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

#scale
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

X_numpy, Y_numpy = datasets.make_regression(n_samples = 100, n_features = 1, noise = 10)

X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))

Y_train = torch.from_numpy(Y_train.astype(np.float32))
Y_test = torch.from_numpy(Y_test.astype(np.float32))

Y_train = Y_train.view(Y_train.shape[0], 1)
Y_test = Y_test.view(Y_test.shape[0], 1)

In [3]:
X.shape

(569, 30)

# Model prediction

In [4]:
input_size = n_features
output_size = 1

class LogisticRegression(nn.Module):
  def __init__(self, input_dim, output_dim):
    super(LogisticRegression, self).__init__()
    #define layers
    self.lin = nn.Linear(input_dim, output_dim)
  def forward(self, x):
    return torch.sigmoid(self.lin(x))


In [5]:
model = LogisticRegression(input_size, output_size)

Loss

In [6]:
loss = nn.BCELoss()

#Training

In [11]:
lr = 0.01
n = 1000

optimizer = torch.optim.SGD(model.parameters(), lr = lr)

for epoch in range(n):
  #prediction
  H = model(X_train)
  #loss
  L = loss(H, Y_train)
  L.backward()
  #update
  optimizer.step()
  #zero gradients
  optimizer.zero_grad()

  if (epoch+1) % 100 == 0:
    [w,b] = model.parameters()
    print(f'epoch = {epoch+1}: w = {w[0][0]:.3f},loss = {L.item():.5f}')
with torch.no_grad():
  y_predicted = model(X_test)
  y_predicted_cls = y_predicted.round()
  accuracy = y_predicted_cls.eq(Y_test).sum()/float(Y_test.shape[0])
  print(f'accuracy = {accuracy:.4f}')

epoch = 100: w = -0.080,loss = 0.23854
epoch = 200: w = -0.138,loss = 0.18313
epoch = 300: w = -0.176,loss = 0.15838
epoch = 400: w = -0.204,loss = 0.14376
epoch = 500: w = -0.226,loss = 0.13388
epoch = 600: w = -0.245,loss = 0.12665
epoch = 700: w = -0.260,loss = 0.12105
epoch = 800: w = -0.273,loss = 0.11656
epoch = 900: w = -0.285,loss = 0.11284
epoch = 1000: w = -0.296,loss = 0.10970
accuracy = 1.0000


#Visualization

In [ ]:
#predictions = model(X).detach().numpy()
#plt.plot(X_numpy, Y_numpy, 'ro')
#plt.plot(X_numpy, predictions, 'b')